In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import string
import pymorphy2
from scipy.sparse import *
# import enchant
from gensim.parsing.preprocessing import *
import stop_words
from nltk.stem.snowball import RussianStemmer
from nltk.tokenize import sent_tokenize, word_tokenize 
from nltk.stem import PorterStemmer, WordNetLemmatizer

%matplotlib notebook

In [2]:
def log_progress(sequence, every=None, size=None, name='Items'):
    from ipywidgets import IntProgress, HTML, VBox
    from IPython.display import display

    is_iterator = False
    if size is None:
        try:
            size = len(sequence)
        except TypeError:
            is_iterator = True
    if size is not None:
        if every is None:
            if size <= 200:
                every = 1
            else:
                every = int(size / 200)     # every 0.5%
    else:
        assert every is not None, 'sequence is iterator, set every'

    if is_iterator:
        progress = IntProgress(min=0, max=1, value=1)
        progress.bar_style = 'info'
    else:
        progress = IntProgress(min=0, max=size, value=0)
    label = HTML()
    box = VBox(children=[label, progress])
    display(box)

    index = 0
    try:
        for index, record in enumerate(sequence, 1):
            if index == 1 or index % every == 0:
                if is_iterator:
                    label.value = '{name}: {index} / ?'.format(
                        name=name,
                        index=index
                    )
                else:
                    progress.value = index
                    label.value = u'{name}: {index} / {size}'.format(
                        name=name,
                        index=index,
                        size=size
                    )
            yield record
    except:
        progress.bar_style = 'danger'
        raise
    else:
        progress.bar_style = 'success'
        progress.value = index
        label.value = "{name}: {index}".format(
            name=name,
            index=str(index or '?')
        )

## RegExp crash course

https://regex101.com

https://regexr.com

In [3]:
match = re.fullmatch(r'\d\d\D\d\d', r'12-12') 
print('YES' if match else 'NO') 

YES


In [4]:
match = re.fullmatch(r'\d\d\D\d\d', r'Т. 12-12') 
print('YES' if match else 'NO') 

NO


In [5]:
print(re.split(r'\W+', 'Где, скажите мне, мои очки??!')) 

['Где', 'скажите', 'мне', 'мои', 'очки', '']


In [6]:
print(re.findall(r'\d\d\.\d\d\.\d{4}', 
                 r'Эта строка написана 21.01.2019, а могла бы и 01.09.2017')) 

['21.01.2019', '01.09.2017']


In [7]:
print(re.sub(r'\d\d\.\d\d\.\d{4}', 
             r'DD.MM.YYYY', 
             r'Эта строка написана 21.01.2019, а могла бы и 01.09.2017')) 

Эта строка написана DD.MM.YYYY, а могла бы и DD.MM.YYYY


In [8]:
re.findall(r'\w+$', 'AV is largest Analytics community of India')

['India']

In [9]:
re.findall(r'\w\w', 'AV is largest Analytics community of India')

['AV',
 'is',
 'la',
 'rg',
 'es',
 'An',
 'al',
 'yt',
 'ic',
 'co',
 'mm',
 'un',
 'it',
 'of',
 'In',
 'di']

In [10]:
re.findall(r'@\w+', 
           'abc.test@gmail.com, xyz@test.in, test.first@analyticsvidhya.com, first.test@rest.biz')

['@gmail', '@test', '@analyticsvidhya', '@rest']

In [11]:
re.findall(r'@\w+.\w+', 
           'abc.test@gmail.com, xyz@test.in, test.first@analyticsvidhya.com, first.test@rest.biz')

['@gmail.com', '@test.in', '@analyticsvidhya.com', '@rest.biz']

In [12]:
re.findall(r'@\w+.(\w+)', 
           'abc.test@gmail.com, xyz@test.in, test.first@analyticsvidhya.com, first.test@rest.biz')

['com', 'in', 'com', 'biz']

In [13]:
re.findall(r'[aeiouAEIOU]\w+', 'AV is largest Analytics community of India')

['AV', 'is', 'argest', 'Analytics', 'ommunity', 'of', 'India']

In [14]:
re.findall(r'\d([A-Z][A-Za-z]+)([A-Z][A-Za-z]+)', 
          '1NoahEmma2LiamOlivia3MasonSophia4JacobIsabella5WilliamAva6EthanMia7MichaelEmily')

[('Noah', 'Emma'),
 ('Liam', 'Olivia'),
 ('Mason', 'Sophia'),
 ('Jacob', 'Isabella'),
 ('William', 'Ava'),
 ('Ethan', 'Mia'),
 ('Michael', 'Emily')]

# Обработка реального текста

In [15]:
EXCLUDE_SYMBOLS_STR = u''.join(['№', '«', 'ђ', '°', '±', '‚', 'ћ', '‰', '…', '»', 'ѓ', 'µ', '·', 'ґ', 'њ', 'ї', 'џ', 'є', '‹',
                                '‡', '†', '¶', 'ќ', '€', '“', 'ў', '§', '„', '”', '\ufeff', '’', 'љ', '›', '•', '—', '‘', 
                                '\x7f', '\xad', '¤', '\xa0'])

In [16]:
GRAMMS = ['NOUN', 'ADJF', 'ADJS', 'PRTF', 'PRTS', 'GRND', 'ADVB']

[**Data**](http://study.mokoron.com/)

---

In [17]:
DATA_PATH = 'corpus.csv'

In [18]:
df = pd.read_csv(DATA_PATH,index_col=0)#, header=None, sep=';',
#                  names=['id', 'tdate', 'tmane', 'ttext', 'ttype', 'trep', 'trtw', 'tfav', 'tstcount', 
#                         'tfol', 'tfrien', 'listcount'])
print(df.shape)

(216, 4)


In [19]:
df.head()

,auto,crime,culture,education
0,"[<div class=""itemFullText"">\r\n<p><em><strong>...","[<div class=""itemFullText"">\r\n<p><em><strong>...","[<div class=""itemFullText"">\r\n<p style=""text-...","[<div class=""itemFullText"">\r\n<p><em><strong>..."
1,"[<div class=""itemFullText"">\r\n<p><em><strong>...","[<div class=""itemFullText"">\r\n<p><em><strong>...","[<div class=""itemFullText"">\r\n<p><strong><em>...","[<div class=""itemFullText"">\r\n<p><em><strong>..."
2,"[<div class=""itemFullText"">\r\n<p><em><strong>...","[<div class=""itemFullText"">\r\n<p><em><strong>...","[<div class=""itemFullText"">\r\n<p><em><strong>...","[<div class=""itemFullText"">\r\n<p><em><strong>..."
3,"[<div class=""itemFullText"">\r\n<p><em><strong>...","[<div class=""itemFullText"">\r\n<p><em><strong>...","[<div class=""itemFullText"">\r\n<p><em><strong>...","[<div class=""itemFullText"">\r\n<p style=""text-..."
4,"[<div class=""itemFullText"">\r\n<p><em><strong>...","[<div class=""itemFullText"">\r\n<p><em><strong>...","[<div class=""itemFullText"">\r\n<p><em><strong>...","[<div class=""itemFullText"">\r\n<p><em><strong>..."


In [20]:
df.dtypes

auto         object
crime        object
culture      object
education    object
dtype: object

---

In [21]:
# corpus, texts = [], []

# for topic in log_progress(df.as_matrix()):
#     for message in topic:
#         sp = message.split('\n')
#         corpus += sp
#         texts += sp
# #     break

# print(len(corpus), len(texts))
# del corpus, texts

In [22]:
corpus = []
for topic in df:
    corpus += df[topic].tolist()
    
answer_corpus = pd.Series(corpus)
answer_corpus = answer_corpus.apply(strip_tags)
answer_corpus = answer_corpus.apply(strip_multiple_whitespaces)
corpus = pd.Series(corpus)
corpus = corpus.apply(lambda x: x.lower())
corpus = corpus.apply(strip_tags)
corpus = corpus.apply(strip_punctuation2)
corpus = corpus.apply(strip_numeric)
corpus = corpus.apply(lambda x: re.sub(r'\w+:\/{2}[\d\w-]+(\.[\d\w-]+)*(?:(?:\/[^\s/]*))*', u' ', x))
corpus = corpus.apply(lambda x: re.sub('«|»', ' ', x))
corpus = corpus.apply(strip_multiple_whitespaces)
corpus = corpus.apply(lambda x: x.partition(' – ')[2])
# corpus = [re.sub(' +' , ' ', doc) for doc in corpus]
corpus.iloc[-1]

'сегодня в городе тольна соберутся победителей ii международного творческого детского фотоконкурса в объятиях природы организаторами масштабного проекта выступают фонд содействия развитию муниципальных образований ассоциация территорий расположения атомных электростанций ао концерн росэнергоатом и венгерская аэс пакш город полярные зори представят виктория серебренникова и екатерина таранникова в течение десяти дней в рамках работы экологического лагеря ребят ждёт увлекательная экскурсионная программа по москве с визитом в экспозиционный центр концерна росэнергоатом и будапеште в тольна российские школьники посетят серию мастер классов от известного профессионального фотохудожника балинта винце берег реки дунай маковые поля рыболовные пруды венгерские деревушки и аэс пакш список мест проведения уроков фотоискусства запланированы и съемки метеоритного дождя итогом экологического лагеря станут не только прекрасные фотографии новые знания приятные впечатления но и дружеские связи между шк

In [23]:
print(len(corpus))
corpus.iloc[-1]

864


'сегодня в городе тольна соберутся победителей ii международного творческого детского фотоконкурса в объятиях природы организаторами масштабного проекта выступают фонд содействия развитию муниципальных образований ассоциация территорий расположения атомных электростанций ао концерн росэнергоатом и венгерская аэс пакш город полярные зори представят виктория серебренникова и екатерина таранникова в течение десяти дней в рамках работы экологического лагеря ребят ждёт увлекательная экскурсионная программа по москве с визитом в экспозиционный центр концерна росэнергоатом и будапеште в тольна российские школьники посетят серию мастер классов от известного профессионального фотохудожника балинта винце берег реки дунай маковые поля рыболовные пруды венгерские деревушки и аэс пакш список мест проведения уроков фотоискусства запланированы и съемки метеоритного дождя итогом экологического лагеря станут не только прекрасные фотографии новые знания приятные впечатления но и дружеские связи между шк

**Уберем ссылки:**

In [24]:
print(len(corpus))

864


**Уберем лишние пробелы:**

In [25]:
corpus = np.asarray([doc.strip().strip('\t').replace('\n', u'') for doc in corpus])
print(len(corpus))

864


In [26]:
corpus[-1]

'сегодня в городе тольна соберутся победителей ii международного творческого детского фотоконкурса в объятиях природы организаторами масштабного проекта выступают фонд содействия развитию муниципальных образований ассоциация территорий расположения атомных электростанций ао концерн росэнергоатом и венгерская аэс пакш город полярные зори представят виктория серебренникова и екатерина таранникова в течение десяти дней в рамках работы экологического лагеря ребят ждёт увлекательная экскурсионная программа по москве с визитом в экспозиционный центр концерна росэнергоатом и будапеште в тольна российские школьники посетят серию мастер классов от известного профессионального фотохудожника балинта винце берег реки дунай маковые поля рыболовные пруды венгерские деревушки и аэс пакш список мест проведения уроков фотоискусства запланированы и съемки метеоритного дождя итогом экологического лагеря станут не только прекрасные фотографии новые знания приятные впечатления но и дружеские связи между шк

**Уберем другие лишние символы:**

In [27]:
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [28]:
regex_punct = re.compile('[%s]' % re.escape(string.punctuation))
regex_dig = re.compile('[%s]' % re.escape(string.digits))
regex_symb = re.compile('[%s]' % re.escape(EXCLUDE_SYMBOLS_STR))
regex_struct = re.compile('[%s]' % string.printable + string.whitespace)
emoji_pattern = re.compile("["
        "\U0001F600-\U0001F64F"  # emoticons
        "\U0001F300-\U0001F5FF"  # symbols & pictographs
        "\U0001F680-\U0001F6FF"  # transport & map symbols
        "\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+")

In [29]:
corpus = [regex_punct.sub('', doc) for doc in corpus]
print(len(corpus))

864


In [30]:
corpus = [regex_dig.sub('', doc) for doc in corpus]
print(len(corpus))

864


In [31]:
corpus = [regex_symb.sub(' ', doc) for doc in corpus]
print(len(corpus))

864


In [32]:
corpus = [regex_struct.sub('', doc) for doc in corpus]
print(len(corpus))

864


In [33]:
corpus = [re.sub(' +' , ' ', doc.strip()) for doc in corpus]
print(len(corpus))

864


In [34]:
corpus[1]

'в мурманске произошло дорожно транспортное происшествие с участием двух пешеходов водитель сбил людей на пешеходном переходе сообщают в региональном гибдд наезд произошёл января в в мурманске в районе дома на проспекте кирова водитель автомобиля renault двигаясь со стороны улицы зеленая в сторону проспекта кольский допустил наезд на двух летних подростков которые переходили проезжую часть по нерегулируемому пешеходному переходу в результате дтп пострадавшие обслужены амбулаторно фото мурманск дтп чп'

In [35]:
# texts = np.asarray([txt for doc, txt in zip(corpus, texts) if len(doc) > 0])
# print(len(texts))
corpus = np.asarray([doc for doc in corpus if len(doc) > 0])
print(len(corpus))

598


In [36]:
corpus = [doc.lower() for doc in corpus]
print(len(corpus))

598


In [37]:
corpus[-1]

'сегодня в городе тольна соберутся победителей ii международного творческого детского фотоконкурса в объятиях природы организаторами масштабного проекта выступают фонд содействия развитию муниципальных образований ассоциация территорий расположения атомных электростанций ао концерн росэнергоатом и венгерская аэс пакш город полярные зори представят виктория серебренникова и екатерина таранникова в течение десяти дней в рамках работы экологического лагеря ребят ждёт увлекательная экскурсионная программа по москве с визитом в экспозиционный центр концерна росэнергоатом и будапеште в тольна российские школьники посетят серию мастер классов от известного профессионального фотохудожника балинта винце берег реки дунай маковые поля рыболовные пруды венгерские деревушки и аэс пакш список мест проведения уроков фотоискусства запланированы и съемки метеоритного дождя итогом экологического лагеря станут не только прекрасные фотографии новые знания приятные впечатления но и дружеские связи между шк

In [38]:
corpus = [emoji_pattern.sub('', doc) for doc in corpus]
print(len(corpus))

598


In [39]:
corpus[100]

'на дороге между колой и туломой в день народного единства улетела в кювет иномарка свидетели происшествия сняли и выложили в социальные сети видео на котором эвакуаторщики пытаются достать автомобиль получается у них это с трудом потому что машина лежит достаточно далеко от дороги эвакуаторщикам приходится не только использовать стрелу но и буквально толкать автомобиль своими плечами видео по ссылке'

In [40]:
import pymorphy2
morph = pymorphy2.MorphAnalyzer()

In [41]:
corpus_tokens = []
inds_to_drop = []

for i, sentence in enumerate(corpus[:]):
    tmp_tokens = []
    sp = sentence.split()
    for word in sp:
        if word not in stop_words.get_stop_words('ru'):
            if morph.word_is_known(word):
                tmp_tokens.append(word)
    if len(tmp_tokens) > 0:
        corpus_tokens.append(tmp_tokens)
    else:
        inds_to_drop.append(i)
#     break
        
print(len(corpus_tokens))#, len(texts))

598


In [42]:
answer_corpus = answer_corpus[~answer_corpus.index.isin(inds_to_drop)]
answer_corpus.index = pd.RangeIndex(len(answer_corpus))

In [43]:
stemmer = RussianStemmer()

*см. также  **лемамтизацию** в [pymorphy2](https://pymorphy2.readthedocs.io/en/latest/user/guide.html#normalization) и [nltk](https://www.nltk.org/api/nltk.stem.html?highlight=lemmatizer#nltk.stem.wordnet.WordNetLemmatizer)*

In [44]:
corpus_tokens_stem = []

for i, tokens in enumerate(log_progress(corpus_tokens[:])):
    tmp = [stemmer.stem(word) for word in tokens]
    corpus_tokens_stem.append(tmp)
#     break
    
print(len(corpus_tokens_stem))

VBox(children=(HTML(value=''), IntProgress(value=0, max=598)))

598


In [45]:
corpus_tokens[100]

['дороге',
 'колой',
 'туломой',
 'народного',
 'единства',
 'улетела',
 'кювет',
 'иномарка',
 'свидетели',
 'происшествия',
 'сняли',
 'выложили',
 'социальные',
 'сети',
 'видео',
 'котором',
 'пытаются',
 'достать',
 'автомобиль',
 'получается',
 'трудом',
 'машина',
 'лежит',
 'достаточно',
 'дороги',
 'приходится',
 'использовать',
 'стрелу',
 'буквально',
 'толкать',
 'автомобиль',
 'своими',
 'плечами',
 'видео',
 'ссылке']

In [46]:
corpus_tokens_stem[100]

['дорог',
 'кол',
 'тулом',
 'народн',
 'единств',
 'улетел',
 'кювет',
 'иномарк',
 'свидетел',
 'происшеств',
 'снял',
 'вылож',
 'социальн',
 'сет',
 'виде',
 'котор',
 'пыта',
 'доста',
 'автомобил',
 'получа',
 'труд',
 'машин',
 'леж',
 'достаточн',
 'дорог',
 'приход',
 'использова',
 'стрел',
 'буквальн',
 'толка',
 'автомобил',
 'сво',
 'плеч',
 'виде',
 'ссылк']

---

In [47]:
corpus_tokens_stem = np.asarray(corpus_tokens_stem)
# texts = np.asarray(texts)

print(len(corpus_tokens_stem))#, len(texts))

598


In [48]:
diffs = np.asarray([len(tokens) for tokens in corpus_tokens_stem])
print(len(diffs))

598


In [49]:
diffs

array([ 30,  51,  55,  48,  48,  77,  25,  77,  37,  45,  25,  41,  45,
        54,  38, 120,  34,  40,  51,  74,  25,  40,  33,  28,  66,  86,
        75,  42,  99,  32,  52,  57,  33, 108, 144,  48,  71,  92, 176,
        39,  90,  45,  50,  76,  55,  32,  54,  25,  32,  37, 119,  31,
        37,  29,   1,  54,  97,  44,  44,  44,  43,  46,  83,  49,  61,
        35,  51,  17,  42,  28,  39, 102,  17,  29,  82,  47, 101,  47,
        35,  79,  27,  35,  49,  39,  51,  78,  38,  38,  37,  63,  60,
        32,  42,  63,  33,  43,  81,  34,  79, 142,  35,  51, 100,  34,
        54,  38,   4,  63,  50,  35, 141,  43,  27,  22,  37,  17,  62,
        38, 223,  65,  52,  92,  38,  63,  50,  38,  31,  14,  45,  37,
        27,  31,  34,  78,  43,  25,  32,  40,  88,  45,  36,  33,  25,
        52,  47, 143,  51,  40,  64,  47,  30,  60,  31,  60,  74,  77,
        33, 101,  91,  81, 100,  48,  48,  51,  88,  72,  57,  93,  99,
        62,  67,  56,  68,  73,  64,  71,  67,  68,  48,  50,  5

In [50]:
plt.figure(figsize=(15, 5))
plt.hist(diffs, bins=100)
plt.grid(True)
plt.xlim(0, 30)

<IPython.core.display.Javascript object>

(0, 30)

In [51]:
corpus_tokens_stem[diffs == 2]

array([list(['вход', 'свободн'])], dtype=object)

In [52]:
np.unique(corpus_tokens_stem).shape[0] / float(corpus_tokens_stem.shape[0])

0.9966555183946488

In [53]:
stat = {}

tmp_corp, tmp_texts = [], []

for i, tokens in enumerate(log_progress(corpus_tokens_stem)):
    s = ' '.join(tokens)
    if stat.get(s, None) is None:
        stat[s] = True
        tmp_corp.append(tokens)
        tmp_texts.append(corpus[i])
        
corpus_tokens_stem = tmp_corp
texts = tmp_texts
print(len(corpus_tokens_stem), len(texts))

VBox(children=(HTML(value=''), IntProgress(value=0, max=598)))

596 596


In [54]:
corpus_tokens_stem = np.asarray(corpus_tokens_stem)
# texts = np.asarray(corpus)

print(len(corpus_tokens_stem))#, len(texts))

596


In [55]:
diffs = np.asarray([len(tokens) for tokens in corpus_tokens_stem])
print(len(diffs))

596


In [56]:
plt.figure(figsize=(15, 5))
plt.hist(diffs, bins=100)
plt.grid(True)
plt.xlim(0, 10)

<IPython.core.display.Javascript object>

(0, 10)

In [57]:
corpus_tokens_stem[diffs == 7]

array([ list(['будьт', 'предельн', 'внимательн', 'общен', 'телефон', 'незнаком', 'людьм'])], dtype=object)

In [58]:
corpus_tokens_stem[diffs >= 8].shape

(586,)

In [59]:
corp4learning = corpus_tokens_stem[diffs >= 8]
# texts4learning = texts[diffs >= 8]

In [60]:
tmp_corp = []

for tokens in corp4learning:
    tmp_corp.append([t for t in tokens if len(t) > 2])
corp4learning = tmp_corp

In [61]:
vocab = np.unique(np.concatenate(corp4learning).flatten()).tolist()
print(len(vocab))

7028


In [62]:
real_vocab = dict.fromkeys(vocab,0)
for text in corp4learning:
    for word in text:
        real_vocab[word]+=1
top_used_vocab = pd.Series(real_vocab)
top_used_vocab.sort_values(ascending=False,inplace=True)
top_used_vocab.head()

мурманск     694
област       268
автомобил    196
работ        182
дел          168
dtype: int64

In [63]:
top100 = top_used_vocab.head(100).values

In [64]:
plt.figure(figsize=(15,10))
plt.bar(top_used_vocab.head(100).index,top_used_vocab.head(100).values)
plt.title('ТОП 100 самых употребляемых слов')
plt.xticks(rotation='vertical');

<IPython.core.display.Javascript object>

### Удалим самые употребляемые

In [65]:
vocab.pop(vocab.index('мурманск'))

'мурманск'

In [66]:
vocab.pop(vocab.index('област'))

'област'

In [67]:
'мурманск' in vocab

False

In [68]:
serialised_vocab = pd.Series(vocab)

In [69]:
serialised_vocab

0           абитуриент
1              абонент
2                  абр
3                абрам
4              абрамов
5             абсолютн
6               абсурд
7             авангард
8                 авар
9              аварийн
10              август
11              авдеев
12           авиабилет
13           авиамодел
14             авиатор
15            авиацион
16               аврор
17             австрал
18                 авт
19             автобус
20            автобусн
21        автовладелец
22        автовладельц
23            автограф
24           автодорог
25       автоинспектор
26         автоинспекц
27         автолюбител
28             автомат
29      автоматизирова
             ...      
6996             юрьев
6997              юсуп
6998             яблок
6999            яблочк
7000               явк
7001               явл
7002              явля
7003               явн
7004             ягодк
7005             ядерн
7006              язык
7007            языков
7008       

In [70]:
corp_matrix = []
for text  in corp4learning:
    text_list = []
    for word in text:
        if word in serialised_vocab.values:
            text_list.append(serialised_vocab[serialised_vocab == word].index.values[0]) 
    corp_matrix.append(text_list)

KeyboardInterrupt: 

In [ ]:
corp_matrix

In [ ]:
serialised_vocab[serialised_vocab == 'кольск'].index.values[0]

In [ ]:
serialised_vocab

In [ ]:
corp_matrix[0]

In [ ]:
corp4learning[0]

In [ ]:
def computeTF(text):
    #print(len(text))
    text_dict = set()
    for word in text:
        if word in serialised_vocab.values:
            #print('added word to dictionary')
            text_dict.add(word)
    text_len = len(text)
    print(text_dict)
    text_dict = dict.fromkeys(text_dict,0)
    for word in text:
        if word in text_dict.keys():
            text_dict[word] += 1
    tf_vec = []
    for word in text_dict.keys():
        tf_vec.append(text_dict[word]/text_len)
    #print(len(tf_vec))
    return tf_vec

In [ ]:
computeTF(corp4learning[0])

In [ ]:
def computeIDF(corpus,text):
    idf_vec = []
    text_dict = set()
    for word in text:
        if word in serialised_vocab.values:
            #print('added word to dictionary')
            text_dict.add(word)
    for word in text_dict:
        if word in serialised_vocab.values:
            idf_vec.append(len(corpus)/top_used_vocab[word])
    #print(len(idf_vec))
    return np.log(idf_vec)

In [ ]:
def computeTF_IDF(corpus,text):
    return computeTF(text)*computeIDF(corpus,text)

In [ ]:
computeTF_IDF(corp4learning,'F')

In [ ]:
TF_IDF_corpus = []
for text in corp4learning:
    TF_IDF_corpus.append(computeTF_IDF(corp4learning,text))

In [ ]:

max_leng = 0
for text in TF_IDF_corpus:
    leng = len(text)
    if (leng > max_leng):
        max_leng = leng
for i in range(len(TF_IDF_corpus)):
    filler = np.zeros(max_leng - len(TF_IDF_corpus[i]))
    TF_IDF_corpus[i] = np.concatenate( (TF_IDF_corpus[i],filler) )


    

In [ ]:
len(corp4learning[0])

In [ ]:
from scipy.sparse import csr_matrix
csr_matrix(TF_IDF_corpus)

In [ ]:
def input_embedding(string):
    string = string.lower()
    string = strip_tags(string)
    string = strip_punctuation2(string)
    string = strip_numeric(string)
    string = re.sub(r'\w+:\/{2}[\d\w-]+(\.[\d\w-]+)*(?:(?:\/[^\s/]*))*', u' ', string)
    string = re.sub('«|»', ' ', string)
    string = strip_multiple_whitespaces(string)
    string = string.split()
    tokens = []
    for word in string:
        if word not in stop_words.get_stop_words('ru'):
            if morph.word_is_known(word):
                tokens.append(stemmer.stem(word))
    #print(tokens)
    embedding = computeTF_IDF(corp4learning,tokens)
    filler = np.zeros(max_leng - len(embedding))
    embedding = np.concatenate( (embedding,filler) )
    embedding = embedding.reshape(1,-1)
#     print(res_str)
    return embedding

In [ ]:
from sklearn.neighbors import NearestNeighbors

X = 'сколько стоит собрать ребенка в школу'
X = input_embedding(X)
nbrs_bicycle = NearestNeighbors(n_neighbors=5, algorithm='auto',metric='cosine').fit(TF_IDF_corpus)
distances, indices = nbrs_bicycle.kneighbors(X)

In [ ]:
indices

In [ ]:
distances

In [ ]:
answer_corpus[113]

In [ ]:
joined_corp4learning = []
for text in corp4learning:
    joined_corp4learning.append(' '.join(text))

In [ ]:
def str_prepr(stringa):
    res_str = stringa.lower()
    res_str = strip_tags(res_str)
    res_str = strip_punctuation2(res_str)
    res_str = strip_numeric(res_str)
    res_str = re.sub(r'\w+:\/{2}[\d\w-]+(\.[\d\w-]+)*(?:(?:\/[^\s/]*))*', u' ', res_str)
    res_str = re.sub('«|»', ' ', res_str)
    res_str =strip_multiple_whitespaces(res_str)
    sp = res_str.split()
    tmp_tokens = []
    for word in sp:
        if word not in stop_words.get_stop_words('ru'):
            if morph.word_is_known(word):
                tmp_tokens.append(stemmer.stem(word))
    res_str = ' '.join(tmp_tokens)
    res_embedding = vectorizer.transform([res_str])
#     print(res_str)
    return res_embedding

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()
vectorizer.fit(joined_corp4learning)
X = vectorizer.transform(joined_corp4learning)

In [ ]:
nbrs_sklearn = NearestNeighbors(metric='cosine',radius=1.0,n_neighbors=10).fit(X)
test_str = 'на кольском проспекте в районе перекрёстка на улице шевченко'
distances, indices = nbrs_sklearn.kneighbors(str_prepr(test_str))

In [ ]:
indices

In [ ]:
def search_engine(string,sklearn = True,bicycle = False):
    if sklearn:
        assert(bicycle == False)
        distances, indices = nbrs_sklearn.kneighbors(str_prepr(string))
    if bicycle:
        assert(sklearn == False)
        distances, indices = nbrs_bicycle.kneighbors(input_embedding(string))
    print('по вашему поисковому запросу: "' + string + '" найдено:')
    i = 1
    for text in answer_corpus[indices[0]]:
        print(str(i) + " текст")
        print(text)
        i+=1
        print('')

        

In [ ]:
search_engine('сколько стоит собрать ребенка в школу',sklearn=False,bicycle=True)